In [303]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import random as rd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from matplotlib.colors import ListedColormap
from sklearn import linear_model
from sklearn import metrics
from scipy.special import expit
from scipy.special import logit

Here I created new data frame and scraped from Goldin website which known well for its auctions of trading cards.

In [304]:
#df = pd.DataFrame()

In [132]:
#Single cards - Basketball 
link = "https://goldin.co/buy/?show_only=Sold%20Items&page=1&number_of_lots=96&Item_Type=Single%2520Cards&Sub_Category=Basketball"
driver =webdriver.Chrome()
driver.get(link)
driver.implicitly_wait(10)
cards = driver.find_elements(By.CSS_SELECTOR,".css-175oi2r.r-1udh08x.r-y46g1k")



In [133]:
count1 = 0
count2 = 1

for i in range(2,105):
    for card in cards:
        driver.implicitly_wait(30)
        title = card.find_element(By.CSS_SELECTOR,".css-1rynq56.r-8akbws.r-krxsd3.r-dnmrzs.r-1udh08x.r-1udbk01").text
        bids = card.find_element(By.CSS_SELECTOR,".css-175oi2r.r-1d09ksm.r-18u37iz").find_elements(By.CSS_SELECTOR,".css-1rynq56")[1].text
        link = card.find_element(By.CSS_SELECTOR, ".css-175oi2r.r-1awozwy.r-1jj8364.r-13qz1uu").get_attribute("href")
        date = card.find_element(By.CSS_SELECTOR,".css-175oi2r.r-1wtj0ep.r-kzbkwu.r-m2pi6t.r-1hvjb8t.r-tskmnb").text
        price = card.find_element(By.CSS_SELECTOR,".css-175oi2r.r-1d09ksm.r-18u37iz").find_elements(By.CSS_SELECTOR,".css-1rynq56")[0].text
        err_date = 0
        err_bids = 0
        try:
            bids = re.findall("([0-9]*) bids", bids)[0]
        except:
            print("Try again in bids...")
            bids = card.find_element(By.CSS_SELECTOR,".css-175oi2r.r-1d09ksm.r-18u37iz").find_elements(By.CSS_SELECTOR,".css-1rynq56")[1].text
            try:
                bids = re.findall("([0-9]*) bids", bids)[0]
            except:
                err_bids = 1
                bids = str(count1) + "," +str(count2) 

        fixed_date = re.findall("(Mon|Tue|Wed|Thu|Fri|Sat|Sun), (.*), (.*) (AM|PM)", date)
        try:
            weekday = fixed_date[0][0]
            fulldate = fixed_date[0][1]
            hour = fixed_date[0][2]
            ampm = fixed_date[0][3]
        except:
            print("try again in date...")
            try:
                date = card.find_element(By.CSS_SELECTOR,".css-175oi2r.r-1wtj0ep.r-kzbkwu.r-m2pi6t.r-1hvjb8t.r-tskmnb").text
                fixed_date = re.findall("(Mon|Tue|Wed|Thu|Fri|Sat|Sun), (.*), (.*) (AM|PM)", date)
                weekday = fixed_date[0][0]
                fulldate = fixed_date[0][1]
                hour = fixed_date[0][2]
                ampm = fixed_date[0][3]
            except:
                err_date = 1
                weekday = str(count1) + ',' + str(count2)
                fulldate = str(count1) + ',' + str(count2)
                hour = str(count1) + ',' + str(count2)
                ampm = str(count1) + ',' + str(count2)

        '''
        print(title)
        print(bids)
        print(link)
        print(price)
        print(weekday, ", ", fulldate, ", ", hour," ", ampm)
        print("======")
        '''
        
        
        row = pd.Series({"title":title,"bids":bids,"err_bids":err_bids,"link": link, "price": price,"weekday":weekday, "date":fulldate,"err_date":err_date, "hour":hour, "ampm": ampm})
        df = pd.concat([df, row.to_frame().T], ignore_index=True)
        count1 = count1 +1
    link = "https://goldin.co/buy/?Item_Type=Single%2520Cards&Sub_Category=Basketball&number_of_lots=96&page=" +  str(i) +"&show_only=Sold%20Items"
    driver.get(link)
    driver.implicitly_wait(30)
    cards = driver.find_elements(By.CSS_SELECTOR,".css-175oi2r.r-1udh08x.r-y46g1k")
    count2 = count2 + 1
    print("count1: " , count1, " count2: " ,  count2 , " in 104")

count1:  0  count2:  2  in 104


In [111]:
#df["Description"] = ''

Dropping duplicates because I did scraping again and again until I got enough records

In [ ]:
#df = pd.read_csv("Basketball1.csv")
#df.drop(df.columns[0], axis=1, inplace=True)
#df.drop(df.columns[0], axis=1, inplace=True)
#df.to_csv("Basketball1.csv")
#df

<a id='save'>Save into CSV file</a>

In [145]:
start = 0
end = 9140
index =10
df.to_csv('Basketball.csv')
df.iloc[start:end,:] 

,title,bids,err_bids,link,price,weekday,date,err_date,hour,ampm,Description,price_cln
0,2009-10 Panini Playoff National Treasures Rook...,14.0,0.0,https://goldin.co/item/2009-10-panini-playoff-...,"$312,000",Thu,3/30/23,0.0,4:56,AM,"Graded MINT 9 by PSA, with Autograph Certified...",312000
1,2008-09 UD Exquisite Collection NBA All Access...,22.0,0.0,https://goldin.co/item/2008-09-ud-exquisite-co...,"$288,000",Thu,3/30/23,0.0,4:30,AM,"One of the hobby's finest modern artifacts, a ...",288000
2,2002-03 Upper Deck NBA Logo Mania Patches Auto...,21.0,0.0,https://goldin.co/item/2002-03-upper-deck-nba-...,"$240,000",Thu,2/09/23,0.0,4:30,AM,Encapsulated by Beckett. Hall of Famer. This i...,240000
3,2003-04 Topps Chrome Gold Refractor #111 LeBro...,18.0,0.0,https://goldin.co/item/2003-04-topps-chrome-go...,"$204,000",Thu,3/30/23,0.0,4:30,AM,Graded GEM MINT 9.5 by BGS (0003719792). Rooki...,204000
4,2003-04 Topps Chrome Gold Refractor #111 LeBro...,27.0,0.0,https://goldin.co/item/2003-topps-chrome-gold-...,"$900,000",Thu,2/09/23,0.0,6:46,AM,Graded GEM MT 10 by PSA (17811919). Rookie Car...,900000
...,...,...,...,...,...,...,...,...,...,...,...,...
9135,1999-00 Upper Deck MJ Athlete Of The Century #...,6.0,0.0,https://goldin.co/item/1999-upper-deck-mj-athl...,$20,Fri,4/28/23,0.0,4:15,AM,No basketball player in hardwood history has c...,20
9136,1999 Upper Deck Century Legends Century Collec...,2.0,0.0,https://goldin.co/item/1999-upper-deck-century...,$11,Fri,4/21/23,0.0,4:15,AM,No Data,11
9137,1991-92 Fleer #29 Michael Jordan – PSA NM-MT 8,2.0,0.0,https://goldin.co/item/1991-fleer-29-michael-j...,$8,Fri,3/17/23,0.0,3:15,AM,No basketball player in hardwood history has c...,8
9138,2002-03 Topps Finest #178 LeBron James Rookie ...,24.0,0.0,https://goldin.co/item/2002-03-topps-finest-17...,$378,Fri,5/05/23,0.0,4:31,AM,Future Hall of Famer LeBron James has dominate...,378


In [82]:
long = len(df)
driver =webdriver.Chrome()
for i in range(start,end):
    if(pd.isnull(df.iloc[i,index]) or df.iloc[i,index]=='NaN' or df.iloc[i,index]==""):
        print(i ," in ", long)
        flag = 1
        try:
            link = df.at[i,"link"]
        except:
            print("Error in ", i)
            flag = 0
        if(flag):
            driver.get(link)
            driver.implicitly_wait(8)
        try:
            des = driver.find_element(By.XPATH,"//div[@class='css-175oi2r'][@id = 'html-description']").text
        except:
            des = 'No Data'
        df.iloc[i,index]=des
print("END")

END


For saveing df into CSV file, I need to do again [here](#save)